<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/qwen3_4B_zero-shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:

# Cargamos el archivo CSV que se sube a Colab.
# Debe contener las columnas input_modelo y output_modelo.
# Cada fila representa un par input-output del corrector.

file_path = "/content/frases_corrector.csv"
df = pd.read_csv(file_path, encoding='latin1', delimiter=';')

In [6]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [7]:
# Cargar modelo e tokenizer (Qwen3-4B-Instruct-2507)
# LLM-as-a-Judge binario para concordancia de número en galego

MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"

print("🔄 Cargando tokenizer...")
if HF_TOKEN:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
if HF_TOKEN:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        token=HF_TOKEN,
        device_map="auto",
        torch_dtype="auto"
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [ ]:
def build_prompt(input_text, output_text):
    return f"""
Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical (GEC) en galego. En concreto, a túa tarefa é a de avaliar a saída dun corrector gramatical e decidir se o modelo GEC corrixiu correctamente o erro de concordancia de número da frase inicial (é dicir, singular/plural entre determinante, substantivo, adxectivo ou verbo).

Recibirás sempre:

input_corrector: a frase orixinal que se lle pasou ao corrector
output_corrector: a frase devolta polo corrector

Devolve a resposta exactamente co seguinte formato, sen texto adicional:

input_corrector: "<oración de entrada do corrector>"
output_corrector: "<oración xa avaliada polo corrector>"
tag: <0 ou 1>
explanation: "<explicación breve e precisa en galego do motivo polo que se escolleu a etiqueta 0 ou a etiqueta 1>"

Criterios:

tag = 1: a saída do corrector non é correcta (segue habendo erro de concordancia de número no output_corrector).

tag = 0: a saída do corrector é correcta (non hai erro de concordancia de número no output_corrector).

Nota sobre a tag = 0:

Caso A: input_corrector ≠ output_corrector.
O input_corrector tiña un erro de concordancia de número e o modelo GEC corrixiuno.
A explicación debe indicar que erro(s) concordancia se arranxou/arranxaron.

Caso B: input_corrector = output_corrector.
O input_corrector xa era correcto respecto á concordancia de número e o modelo GEC non fixo cambios adicionais porque non había nada que corrixir.
A explicación debe indicar que non había erro de concordancia de número no input.

Restricións:

Non debes, baixo ningún concepto, corrixir ou modificar de ningunha forma o input nin o output do corrector.
Tes que limitarte exclusivamente a decidir se o erro de concordancia de número foi corrixido polo modelo GEC ou non.
Non debes avaliar outros tipos de erros.

Agora, avalía o seguinte caso:

input_corrector: "{input_text}"
output_corrector: "{output_text}"
"""


# Recorremos el CSV fila por fila.
# Para cada par input_modelo / output_modelo construimos el prompt
# y generamos la respuesta del modelo.

results = []

for idx, row in df.iterrows():
    input_text = str(row["input_modelo"])
    output_text = str(row["output_modelo"])

    prompt = build_prompt(input_text, output_text)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0,
            do_sample=False
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    results.append(decoded)


# Añadimos las evaluaciones generadas como una nueva columna
# y guardamos el resultado en un nuevo archivo Excel.

df["avaliacion_qwen"] = results
df.to_excel("/content/resultados_qwen.xlsx", index=False)

print("Evaluación completada y guardada en /content/resultados_qwen.xlsx")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
